In [41]:
!pip install cplex --trusted-host pypi.org --trusted-host pypi.python.org --trusted-host=files.pythonhosted.org

!pip install docplex --trusted-host pypi.org --trusted-host pypi.python.org --trusted-host=files.pythonhosted.org

In [42]:
import sys
import docplex.mp as mp
from docplex.mp.model import Model

In [43]:
worker_types = ["Line Workers", "Technicians", "Supervisors", "Engineers"]
candy_types = ["Fruity Fun", "Chocolate Craze", "Tis the Season"]

# i, j, k, l
num_workers = 4
num_plants = 4
num_months = 12
num_candies = 3

# worker type demand data by month
plant_1_demand = [[50, 58, 40, 35, 30, 37, 45, 48, 35, 36, 40, 44],
                 [6, 8, 5, 5, 4, 5, 6, 6, 7, 8, 7, 6],
                 [10, 9, 10, 12, 15, 15, 10, 8, 9, 9, 16, 12],
                 [7, 9, 6, 8, 6, 5, 8, 7, 6, 7, 6, 4]]
plant_2_demand = [[30, 38, 40, 36, 40, 37, 45, 48, 43, 36, 43, 38],
                 [7, 6, 7, 6, 5, 6, 5, 6, 5, 7, 8, 5],
                 [13, 15, 12, 12, 11, 11, 10, 8, 9, 10, 12, 12],
                 [8, 8, 7, 9, 6, 6, 7, 6, 5, 5, 6, 3]]
plant_3_demand = [[40, 48, 40, 35, 33, 27, 38, 38, 32, 26, 45, 42],
                 [6, 6, 7, 5, 7, 5, 6, 5, 6, 7, 6, 4],
                 [15, 15, 15, 13, 11, 15, 12, 9, 9, 11, 11, 10],
                 [7, 8, 8, 8, 7, 6, 8, 8, 6, 8, 5, 5]]
plant_4_demand = [[50, 55, 45, 45, 40, 32, 35, 42, 30, 32, 47, 40],
                 [6, 6, 6, 5, 4, 7, 7, 7, 9, 8, 7, 7,],
                 [12, 12, 13, 12, 11, 15, 8, 9, 10, 11, 11, 12],
                 [7, 8, 7, 6, 6, 9, 8, 7, 7, 7, 7, 6]]

# cost per type of worker
worker_monthly_cost = [4200, 4800, 5000, 6600]

# annual maintenance cost per plant per candy
fixed_cost = [[35000, 40000, 41000],
             [28000, 30000, 27000],
             [25000, 27000, 28000],
             [30000, 35000, 29000]]   
        

In [46]:
class Project_One:
    def __init__(self):
        self.m = Model(name='OR II Project 1')
        
    def create_variables(self):
        
        plants = 4
        workers = 4
        months = 12
        candies = 3
        
        # staff_plan [worker][plant][month]
        self.staff_plan = []
        
        for _ in range(num_workers):
            worker_assignment = []
            for _ in range(num_plants):
                plant_assignment = []
                for _ in range(num_months):
                    plant_assignment.append(self.m.integer_var())
                worker_assignment.append(plant_assignment)
            self.staff_plan.append(worker_assignment)
            
        # binary variable to for whether or not plant j produces candy l
        # y [plant][candy]
        self.y = []
        
        for _ in range(num_plants):
            plant_product = []
            for _ in range(num_candies):
                plant_product.append(self.m.binary_var())
            self.y.append(plant_product)
            
    def create_objective_function(self):
        
        self.m.minimize(sum(self.staff_plan[i][j][k]*worker_monthly_cost[i] 
                            for i in range(num_workers) 
                            for j in range(num_plants) 
                            for k in range(num_months))
                        +(sum(self.y[j][l]*fixed_cost[j][l] 
                            for j in range(num_plants) 
                            for l in range(num_candies))))
        
    def create_constraints(self):
        
        # At least 3 engineers, 6 supervisors, and 2 technicians per candy produced at a plant
        for j in range(num_plants):
                for k in range(num_months):
                    self.m.add_constraint(self.staff_plan[1][j][k] >= 2*sum(self.y[j][l] for l in range(num_candies)))
                    self.m.add_constraint(self.staff_plan[2][j][k] >= 6*sum(self.y[j][l] for l in range(num_candies)))
                    self.m.add_constraint(self.staff_plan[3][j][k] >= 3*sum(self.y[j][l] for l in range(num_candies)))
                    
        # Workers per month must not change more than 20%
        for j in range(num_plants):
                for i in range(num_workers):
                    for k in range(1, 12):
                        self.m.add_constraint(self.staff_plan[i][j][k] <= 1.2*self.staff_plan[i][j][k-1])
                        self.m.add_constraint(self.staff_plan[i][j][k] >= 0.8*self.staff_plan[i][j][k-1])
        
        # Must meet monthly demand estimates
        for i in range(num_workers):
            for k in range(num_months):
                for l in range(num_candies):
                    self.m.add_constraint(self.staff_plan[i][0][k] >= plant_1_demand[i][k])
                    self.m.add_constraint(self.staff_plan[i][1][k] >= plant_2_demand[i][k])
                    self.m.add_constraint(self.staff_plan[i][2][k] >= plant_3_demand[i][k])
                    self.m.add_constraint(self.staff_plan[i][3][k] >= plant_4_demand[i][k])
                    
        # Fruity Fun either Plant 1 or Plant 2
        self.m.add_constraint(self.y[0][0] + self.y[1][0] <= 1)
        
        # If Chocolate Craze at Plant 2, must be also at Plant 4
        self.m.add_constraint(self.y[1][1] <= self.y[3][1])
        
        # Either Plant 1 or Plant 4 or both produce Tis the Season
        self.m.add_constraint(self.y[0][2] + self.y[3][2] >= 1)
        
        # Each candy must be produced by at least two plants
        for l in range(num_candies):
            self.m.add_constraint(self.y[0][l]+self.y[1][l]+self.y[2][l]+self.y[3][l] >= 2)
            
    def solve_model(self):
        solution = self.m.solve()
        
        if solution:
            p=0
            for workers in self.staff_plan:
                print(f'{worker_types[p]}: ', end="")
                print()
                print()
                q=1
                for plants in workers:
                    print(f"     Plant {q}: ", end="")
                    for months in plants:
                        print(int(months.solution_value), end=" ")
                    print()
                    q+=1
                print()
                p+=1
                
        # tried this block of code before, but didn't realize it had all my info basically transposed
        # if solution:
           # i=1
           # for plants in self.staff_plan:
           #     print(f"Plant {i} ")
           #     j=0
           #     for workers in plants:
           #         print(f'{worker_types[j]}: ', end="")
           #         for months in workers:
           #            print(int(months.solution_value), end=",")
           #         print()
           #         j+=1
           #     print()
           #     i+=1

            
            q=1
            for plants in self.y:
                print(f"Plant {q}: ", end="")
                r = 0
                for candy in plants:
                    print(int(candy.solution_value), end=" ")
                    r+=1
                print()
                q+=1
        print()
        print(f"Cost = ${self.m.objective_value:.2f}")
        #print(solution)

In [47]:
solver = Project_One()
solver.create_variables()
solver.create_objective_function()
solver.create_constraints()
solver.solve_model()

Line Workers: 

     Plant 1: 50 58 47 38 32 38 45 48 39 36 40 44 
     Plant 2: 32 38 40 36 40 38 45 48 43 36 43 38 
     Plant 3: 40 48 40 35 33 32 38 38 32 38 45 42 
     Plant 4: 50 55 45 45 40 32 35 42 34 40 47 40 

Technicians: 

     Plant 1: 7 8 7 6 5 5 6 6 7 8 7 6 
     Plant 2: 7 6 7 6 5 6 5 6 6 7 8 7 
     Plant 3: 6 6 7 6 7 6 6 5 6 7 6 5 
     Plant 4: 6 6 6 5 6 7 7 8 9 8 7 7 

Supervisors: 

     Plant 1: 10 10 11 13 15 15 12 10 12 14 16 13 
     Plant 2: 13 15 12 12 11 11 10 8 9 10 12 12 
     Plant 3: 15 15 15 13 13 15 12 12 12 12 12 12 
     Plant 4: 12 12 13 12 13 15 12 12 12 12 12 12 

Engineers: 

     Plant 1: 8 9 8 8 7 7 8 7 6 7 6 5 
     Plant 2: 8 8 8 9 8 7 7 6 5 5 6 5 
     Plant 3: 7 8 8 8 7 7 8 8 7 8 7 6 
     Plant 4: 7 8 7 7 8 9 8 7 7 7 7 6 

Plant 1: 1 0 0 
Plant 2: 0 0 1 
Plant 3: 1 1 0 
Plant 4: 0 1 1 

Cost = $15126800.00
